In [2]:
import datetime
from pathlib import Path  # 보조 파일의 로컬 경로를 보다 쉽게 ​​처리하는 데 사용됩니다.

import pyecma376_2  # The base library for Open Packaging Specifications. We will use the OPCCoreProperties class.
from basyx.aas import model
from basyx.aas.adapter import aasx

In [3]:
# step 1: File 객체를 사용하여 SupplementaryFileContainer 및 AAS 및 하위 모델 설정
# step 2: AASX 패키지에 AAS 개체 및 보조 파일 쓰기
# step 3: AASX 패키지에서 AAS 개체 및 보조 파일 읽기


########################################################################################
# Step 1: File 객체를 사용하여 SupplementaryFileContainer 및 AAS 및 하위 모델 설정 #
########################################################################################

# 먼저 간단한 하위 모델을 사용하여 기본 자산 관리 셸을 만들어 보겠습니다.
# 자세한 내용은 `tutorial_create_simple_aas.py`를 참조하세요.

submodel = model.Submodel(
    id_='https://acplt.org/Simple_Submodel'
)
aas = model.AssetAdministrationShell(
    id_='https://acplt.org/Simple_AAS',
    asset_information=model.AssetInformation(
        asset_kind=model.AssetKind.INSTANCE,
        global_asset_id='http://acplt.org/Simple_Asset'
    ),
    submodel={model.ModelReference.from_referable(submodel)}
)

# AAS와 관련되지 않은 또 다른 하위 모델:
unrelated_submodel = model.Submodel(
    id_='https://acplt.org/Unrelated_Submodel'
)

# ID로 쉽게 검색할 수 있도록 이러한 객체를 ObjectStore에 추가합니다.
# 자세한 내용은 `tutorial_storage.py`를 참조하세요. 여기서는 데이터베이스 지원 ObjectStore를 사용할 수도 있습니다.
# (see `tutorial_backend_couchdb.py`).
object_store = model.DictObjectStore([submodel, aas, unrelated_submodel])


# AASX 패키지에 최종적으로 추가될 보조 파일을 보관하려면 SupplementalFileContainer가 필요합니다.
# `DictSupplementalFileContainer`는 파일 내용을 메모리의 단순 바이트 객체에 저장하는 간단한 보충파일 컨테이너입니다.
file_store = aasx.DictSupplementaryFileContainer()

# 이제 로컬 파일 시스템의 예제 파일을 보충파일컨테이너에 추가합니다.
# 이 목적을 위해서는 보충파일컨테이너에 파일 이름을 지정해야 합니다. 
# 이 이름은 컨테이너의 파일을 참조하는 데 사용되며 나중에 AASX 패키지 파일에서 파일 이름으로 사용됩니다. 
# 따라서 이 파일은 슬래시로 시작해야 하며 `/aasx/`로 시작해야 합니다. 여기서는 `/aasx/suppl/MyExampleFile.pdf`를 사용합니다.
# ComponentFileContainer의 add_file() 메소드는 내용이 다르지만 동일한 이름의 파일이 존재하는 경우 접미사를 추가하여 이름의 고유성을 보장합니다.
# 최종 이름이 반환됩니다. 또한 나중에 AASX 패키지의 메타데이터에 사용되는 파일의 MIME 유형을 지정해야 합니다. 
# (이것은 실제로 패키지 내 모든 단일 파일의 MIME 유형("콘텐츠 유형") 사양을 적용하는 기본 개방형 패키징 규칙(ECMA376-2) 형식의 요구 사항입니다.

with open(Path(__file__).parent / 'data' / 'TestFile.pdf', 'rb') as f:
    actual_file_name = file_store.add_file("/aasx/suppl/MyExampleFile.pdf", f, "application/pdf")

NameError: name '__file__' is not defined

In [4]:
Path(__file__)

NameError: name '__file__' is not defined

In [6]:
with aasx.AASXWriter("MyAASXPackage.aasx") as writer:
    # Write the AAS and everything belonging to it to the AASX package
    # The `write_aas()` method will automatically fetch the AAS object with the given id
    # and all referenced Submodel objects from the ObjectStore. It will also scan every object for
    # semanticIds referencing ConceptDescription, fetch them from the ObjectStore, and scan all sbmodels for `File`
    # objects and fetch the referenced auxiliary files from the SupplementaryFileContainer.
    # In order to add more than one AAS to the package, we can simply add more Identifiers to the `aas_ids` list.
    #
    # ATTENTION: As of Version 3.0 RC01 of Details of the Asset Administration Shell, it is no longer valid to add more
    # than one "aas-spec" part (JSON/XML part with AAS objects) to an AASX package. Thus, `write_aas` MUST
    # only be called once per AASX package!
    writer.write_aas(aas_ids=['https://acplt.org/Simple_AAS'],
                     object_store=object_store,
                     file_store=file_store)

    # Alternatively, we can use a more low-level interface to add a JSON/XML part with any Identifiable objects (not
    # only an AAS and referenced objects) in the AASX package manually. `write_aas_objects()` will also take care of
    # adding referenced auxiliary files by scanning all submodel objects for contained `File` objects.
    #
    # ATTENTION: As of Version 3.0 RC01 of Details of the Asset Administration Shell, it is no longer valid to add more
    # than one "aas-spec" part (JSON/XML part with AAS objects) to an AASX package. Thus, `write_all_aas_objects` SHALL
    # only be used as an alternative to `write_aas` and SHALL only be called once!
    objects_to_be_written: model.DictObjectStore[model.Identifiable] = model.DictObjectStore([unrelated_submodel])
    writer.write_all_aas_objects(part_name="/aasx/my_aas_part.xml",
                                 objects=objects_to_be_written,
                                 file_store=file_store)

    # We can also add a thumbnail image to the package (using `writer.write_thumbnail()`) or add metadata:
    meta_data = pyecma376_2.OPCCoreProperties()
    meta_data.creator = "Chair of Process Control Engineering"
    meta_data.created = datetime.datetime.now()
    writer.write_core_properties(meta_data)

# Closing the AASXWriter will write some required parts with relationships and MIME types to the AASX package file and
# close the package file afterward. Make sure, to always call `AASXWriter.close()` or use the AASXWriter in a `with`
# statement (as a context manager) as shown above.


########################################################################
# Step 3: Reading AAS objects and auxiliary files from an AASX package #
########################################################################

# Let's read the AASX package file, we have just written.
# We'll use a fresh ObjectStore and SupplementaryFileContainer to read AAS objects and auxiliary files into.
new_object_store: model.DictObjectStore[model.Identifiable] = model.DictObjectStore()
new_file_store = aasx.DictSupplementaryFileContainer()

# Again, we need to use the AASXReader as a context manager (or call `.close()` in the end) to make sure the AASX
# package file is properly closed when we are finished.
with aasx.AASXReader("MyAASXPackage.aasx") as reader:
    # Read all contained AAS objects and all referenced auxiliary files
    reader.read_into(object_store=new_object_store,
                     file_store=new_file_store)

    # We can also read the metadata
    new_meta_data = reader.get_core_properties()

    # We could also read the thumbnail image, using `reader.get_thumbnail()`



In [9]:
import zipfile
import xml.etree.ElementTree as ET
import os

In [12]:
def extract_xml_from_aasx(aasx_file_path):
    # ZIP 아카이브로 AASX 파일 열기
    with zipfile.ZipFile(aasx_file_path, 'r') as aasx_zip:
        # ZIP 파일 내의 파일 목록 출력
        print("ZIP 파일 내의 파일 목록:")
        for file_name in aasx_zip.namelist():
            print(file_name)

        # XML 파일 추출 (일반적으로 Main.xml)
        xml_file_name = 'aasx/Injection_Molding_Machine/Injection_Molding_Machine.aas.xml'  # AASX 파일 구조에 따라 다름
        with aasx_zip.open(xml_file_name) as xml_file:
            tree = ET.parse(xml_file)
            root = tree.getroot()
            return root

# AASX 파일 경로
current_dir = os.getcwd()
aasx_file_name = 'aasx_file/Injection_Molding_Machine.aasx'
aasx_file_path = os.path.join(current_dir, aasx_file_name)

# XML 데이터 파싱
root = extract_xml_from_aasx(aasx_file_path)


ZIP 파일 내의 파일 목록:
aasx/aasx-origin
aasx/Injection_Molding_Machine/Injection_Molding_Machine.aas.xml
_rels/.rels
aasx/_rels/aasx-origin.rels
aasx/Injection_Molding_Machine/_rels/Injection_Molding_Machine.aas.xml.rels
aasx/files/2023-10-05_A5_de2f52d5.pdf
[Content_Types].xml
Injection_Molding_Machine.png
aasx/Quality_P.png


In [17]:
import os
import zipfile
import xml.etree.ElementTree as ET
import json

current_dir = os.getcwd()
aasx_file_name = 'aasx_file/AAS_1.aasx'
aasx_file_path = os.path.join(current_dir, aasx_file_name)

print(f"AASX 파일 경로: {aasx_file_path}")

def extract_xml_from_aasx(aasx_file_path):
    # ZIP 아카이브로 AASX 파일 열기
    with zipfile.ZipFile(aasx_file_path, 'r') as aasx_zip:
        # ZIP 파일 내의 파일 목록 출력
        print("ZIP 파일 내의 파일 목록:")
        for file_name in aasx_zip.namelist():
            print(file_name)

        # XML 파일 추출
        # 일반적으로 'aasx/aasx.xml' 또는 'aasx/Main.xml'이 사용됩니다.
        xml_file_name = None
        for file_name in aasx_zip.namelist():
            if file_name.endswith('.xml'):
                xml_file_name = file_name
                print(xml_file_name)
                break
        
        if xml_file_name is None:
            raise FileNotFoundError("XML 파일을 찾을 수 없습니다.")

        print(f"XML 파일 경로: {xml_file_name}")
        
        with aasx_zip.open(xml_file_name) as xml_file:
            tree = ET.parse(xml_file)
            root = tree.getroot()
            return root

def xml_to_dict(element):
    """재귀적으로 XML을 파싱하여 Python 딕셔너리로 변환"""
    node = {
        "tag": element.tag,
        "attrib": element.attrib,
        "text": element.text.strip() if element.text else "",
        "children": [xml_to_dict(child) for child in element]
    }
    return node

# XML 데이터 파싱
root = extract_xml_from_aasx(aasx_file_path)

# XML 데이터를 JSON 형식의 딕셔너리로 변환
parsed_data = xml_to_dict(root)

# JSON 파일로 저장
json_file_path = os.path.join(current_dir, 'output.json')
with open(json_file_path, 'w', encoding='utf-8') as json_file:
    json.dump(parsed_data, json_file, ensure_ascii=False, indent=4)

print(f"JSON 파일이 생성되었습니다: {json_file_path}")


AASX 파일 경로: c:\Users\PC\Desktop\git_repo\ML-DL-Training\AAS\aasx_file/AAS_1.aasx
ZIP 파일 내의 파일 목록:
aasx/aasx-origin
aasx/Injecion_Molding_Machine/Injecion_Molding_Machine.aas.xml
_rels/.rels
aasx/_rels/aasx-origin.rels
[Content_Types].xml
aasx/Injecion_Molding_Machine/Injecion_Molding_Machine.aas.xml
XML 파일 경로: aasx/Injecion_Molding_Machine/Injecion_Molding_Machine.aas.xml
JSON 파일이 생성되었습니다: c:\Users\PC\Desktop\git_repo\ML-DL-Training\AAS\output.json
